In [22]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta
import geopandas as gpd
from mapbox import Uploader
import sys
from urllib import request
from urllib.request import urlopen

In [2]:
sys.path.append('..')

In [3]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])
mapbox_api=os.environ['MAPBOX_API_KEY']
mapbox_up=os.environ['MAPBOX_API_UPLOAD_KEY']

In [4]:
#usar en caso de requerir borrar la tabla para cargar todo de nuevo
#cnx.execute('DROP TABLE IF EXISTS "movilidad"."isci_entel";')

In [5]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
url='https://matiascerda.carto.com/api/v2/sql?format=CSV&filename=datos-movilidad-isci.csv&q=select%20*%20from%20data_variacion'

In [7]:
name='isci_entel'
schema='movilidad'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')

schema movilidad does not exist, creating it


In [8]:
sql="select * from geo.zonas_censales"
zonas= gpd.GeoDataFrame.from_postgis(sql,con=cnx, geom_col='geometry')
zonas['GEOCODIGO']=zonas['GEOCODIGO'].astype(str).str.strip()
zonas['PERSONAS_6_A_64_AÑOS']=zonas['PERSONAS_6_A_14_AÑOS']+zonas['PERSONAS_15_A_64_AÑOS']
crs=zonas.crs

In [10]:
if cnx.has_table("isci_entel",schema="movilidad"):

    #request = request(url)
    csv_len = len(urlopen(url).readlines())
    print('rows in csv url:' +str(csv_len))
    
    sql_len=cnx.execute('SELECT COUNT(*) FROM "'+schema+'"."'+name+'";')
    sql_len=sql_len.scalar()
    print('rows in sql table :'+str(sql_len))
    
    rows=csv_len-sql_len
    print('rows '+str(rows))
    
    if rows>1:
        print('reading last '+str(rows)+' from csv in url')
        # The row indices to skip - make sure 0 is not included to keep the header!
        skip_idx = [x for x in range(rows, csv_len) if x != 0]
        
        # Read the data
        ISCIentel=pd.read_csv(url,skiprows=skip_idx,encoding='utf-8')
        if_exists='append'
    else:
        print('no data to update from url')
        #ISCIentel=pd.read_csv(url,encoding='utf-8')
        if_exists='fail'
else:
    print('reading all data from url')
    ISCIentel=pd.read_csv(url,encoding='utf-8')
    if_exists='replace'

<ipython-input-10-fc2c1a33befb>:1: SADeprecationWarning: The Engine.has_table() method is deprecated and will be removed in a future release.  Please refer to Inspector.has_table(). (deprecated since: 1.4)
  if cnx.has_table("isci_entel",schema="movilidad"):


reading all data from url


In [12]:
if_exists

'replace'

In [13]:
if if_exists!='fail':
    #diferencias entre entradas y salidas de zonas censales estimadas por ISCI-entel Ocean
    #ISCIentel=pd.read_csv(url,encoding='utf-8')
    #diferencias entre entradas
    ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
    ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']].astype(int)
    #diferencias entre salidas
    ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']]=ISCIentel['dif_salida'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
    ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']]=ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']].astype(int)
    ISCIentel['zona_censal']=ISCIentel['zona_censal'].astype(str).str.strip()
    ISCIentel=ISCIentel.drop(['the_geom','cartodb_id','the_geom_webmercator'],axis=1)
    ISCIentel=ISCIentel.merge(zonas,right_on='GEOCODIGO',left_on='zona_censal',how='left')
    ISCIentel['year']='2020'
    ISCIentel.loc[ISCIentel['week']>53,'year']='2021'
    ISCIentel.loc[ISCIentel['week']>53,'week']=ISCIentel['week']-53
    ISCIentel['date']=pd.to_datetime(ISCIentel['year']+'-'+ISCIentel['week'].astype(str)+'-6',format='%Y-%W-%w')
    ISCIentel['salidas']=(ISCIentel['dif_salida_lim_sup']-ISCIentel['dif_salida_lim_inf'])/2+ISCIentel['dif_salida_lim_inf']
    ISCIentel['PERSONAS_MOVILIZADAS']=ISCIentel['PERSONAS_6_A_64_AÑOS']+ISCIentel['PERSONAS_6_A_64_AÑOS']/100*ISCIentel['salidas']

<ipython-input-13-1fe7f7b232c3>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
<ipython-input-13-1fe7f7b232c3>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
<ipython-input-13-1fe7f7b232c3>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ISCIentel[['d

In [14]:
if if_exists!='fail':
    #Variabilidad PONDERADA
    pop_comuna=pd.DataFrame(ISCIentel.groupby(['week','comuna'])['PERSONAS_6_A_64_AÑOS'].sum())
    pop_comuna=pd.DataFrame(pop_comuna.reset_index().groupby('comuna')['PERSONAS_6_A_64_AÑOS'].mean()).reset_index()
    pop_comuna=pop_comuna.rename(columns={'PERSONAS_6_A_64_AÑOS':'PERSONAS_6_A_64_AÑOS_COMUNA'})
    pop_comuna=pop_comuna.reset_index()
    #ISCIentel['PERSONAS_15_A_64_AÑOS_COMUNA']
    ISCIentel=ISCIentel.merge(pop_comuna,on='comuna',how='left')
    #% de personas 15 a 64 años comunal por zona censal
    ISCIentel['PERSONAS_6_A_64_AÑOS_FACTOR']=ISCIentel['PERSONAS_6_A_64_AÑOS']/ISCIentel['PERSONAS_6_A_64_AÑOS_COMUNA']
    ISCIentel['salidas_comunal_ponderada']=ISCIentel['salidas']*ISCIentel['PERSONAS_6_A_64_AÑOS_FACTOR']

In [15]:
if if_exists!='fail':
    ISCIentel = gpd.GeoDataFrame(ISCIentel)
    ISCIentel.set_crs(crs)

In [16]:
if if_exists!='fail':
    ISCIentel['geometry']=ISCIentel['geometry'].centroid
    ISCIentel['Lon']=ISCIentel['geometry'].x
    ISCIentel['Lat']=ISCIentel['geometry'].y
    ISCIentel=ISCIentel.drop('geometry',axis=1)
    ISCIentel=pd.DataFrame(ISCIentel)

<ipython-input-16-fe51752aedeb>:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ISCIentel['geometry']=ISCIentel['geometry'].centroid


In [46]:
ISCIentel['Region Metropolitana']='Resto de Chile'
ISCIentel.loc[ISCIentel['REGION']=='13','Region Metropolitana']='Region Metropolitana'

In [15]:
#for r in ISCIentel['REGION'].unique():
#    name='isci_entel'+str(r)
#    print('saving table '+name)
    #ISCIentel[ISCIentel['REGION']==r].to_postgis(name=name+'_'+str(r), schema=schema,con=cnx,if_exists='replace')
#    ISCIentel[ISCIentel['REGION']==r].to_sql(name, schema=schema,con=cnx,if_exists='replace')

In [16]:
#geojson=ISCIentel[ISCIentel['REGION']=='13']
#geojson['date']=geojson['date'].astype(str)
#geojson=geojson.to_json()

In [17]:
#ISCIentel[ISCIentel['REGION']=='13'].to_file("../ETLcache/mov_temp.geojson", driver='GeoJSON')

In [18]:
#with open("../ETLcache/mov_temp.geojson", 'rb') as src:
#    upload_resp = service.upload(src, 'test_dataset')
#upload_resp
#upload_id = upload_resp.json()['id']

In [47]:
grouped=ISCIentel.groupby('Region Metropolitana')

In [49]:
for region,df in grouped:
    print('Guardando región: '+region)
    name='isci_entel_'+str(region).replace(' ','_')
    print('Escribiendo tabla: '+name+' schema: '+schema)
    df.to_sql(name, schema=schema,con=cnx,if_exists=if_exists)

Guardando región: Region Metropolitana
Escribiendo tabla: isci_entel_Region_Metropolitana schema: movilidad
Guardando región: Resto de Chile
Escribiendo tabla: isci_entel_Resto_de_Chile schema: movilidad


In [50]:
#if if_exists!='fail':
#    ISCIentel.to_sql('isci_entel', schema=schema,con=cnx,if_exists=if_exists)

In [20]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_movilidad.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/movilidad.ipynb')

0